# Serializer Demonstration

This Jupyter notebook expects there to be a database available. Remember to run migrations!

Additionally, `djangorestframework-xml` must be installed for this to work correctly. We do not use the package in the rest of the code, so manual installation is required.

    pip install 'djangorestframework-xml>=1.3.0' 'defusedxml>=0.5.0'

In [5]:
!pip install djangorestframework-xml defusedxml
!pip install defusedxml

In [10]:
import os

In [11]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [16]:
from datetime import date
from pprint import pprint
from xml.dom.minidom import parseString as xml_parse

from rest_framework.renderers import JSONRenderer
#from rest_framework_xml.renderers import XMLRenderer

from organizer.models import NewsLink, Startup, Tag
from organizer.serializers import NewsLinkSerializer, StartupSerializer, TagSerializer

## Simple Example with Tag Object

In [17]:
new_tag = Tag.objects.create(name='test')

IntegrityError: (1062, "Duplicate entry 'test' for key 'organizer_tag.name'")

In [23]:
s_tag = TagSerializer(new_tag)

In [24]:
s_tag

TagSerializer(<Tag: test>):
    url = HyperlinkedIdentityField(lookup_field='slug', view_name='api-tag-detail')
    name = CharField(max_length=31, validators=[<UniqueValidator(queryset=Tag.objects.all())>])
    slug = SlugField(allow_unicode=False, help_text='A Label for URL Config.', max_length=31, validators=[<UniqueValidator(queryset=Tag.objects.all())>])

In [20]:
s_tag.data

AssertionError: `HyperlinkedIdentityField` requires the request in the serializer context. Add `context={'request': request}` when instantiating the serializer.

In [25]:
JSONRenderer().render(s_tag.data)

AssertionError: `HyperlinkedIdentityField` requires the request in the serializer context. Add `context={'request': request}` when instantiating the serializer.

In [26]:
def render_json(serialized_object):
    """Shortcut to make this notebook easier to read"""
    print(
        JSONRenderer().render(
            serialized_object.data,
            accepted_media_type='application/json; indent=4',
        ).decode('utf8')
    )

In [27]:
render_json(s_tag)

AssertionError: `HyperlinkedIdentityField` requires the request in the serializer context. Add `context={'request': request}` when instantiating the serializer.

In [9]:
XMLRenderer().render(s_tag.data)

'<?xml version="1.0" encoding="utf-8"?>\n<root><id>1</id><name>django</name><slug>django</slug></root>'

In [10]:
def render_xml(serialized_object):
    """Shortcut to make this notebook easier to read
    
    If you need serious XML handling, compare 
    LXML to Python's std-lib XML capabilities.
    """
    print(
        xml_parse(  # python std-lib
            XMLRenderer().render( # Django Rest Framework
                serialized_object.data
            )  
        ).toprettyxml()
    )

In [11]:
render_xml(s_tag)

<?xml version="1.0" ?>
<root>
	<id>1</id>
	<name>django</name>
	<slug>django</slug>
</root>



## Serialize Object with Relationships

In [12]:
jambon = Startup.objects.create(
    name='JamBon Software',
    slug='jambon-software',
    description='Software Consulting & Training for Web and Mobile Products',
    founded_date=date(2013, 1, 18),
    contact='django@jambonsw.com',
    website='https://www.jambonsw.com',
)

In [13]:
jambon.tags.add(new_tag, Tag.objects.create(name='web'))

In [14]:
s_jambon = StartupSerializer(jambon)

In [15]:
s_jambon

StartupSerializer(<Startup: JamBon Software>):
    id = IntegerField(read_only=True)
    name = CharField(max_length=31)
    slug = SlugField(allow_blank=True, max_length=31)
    description = CharField()
    founded_date = DateField()
    contact = EmailField()
    website = URLField(max_length=255)
    tags = TagSerializer(many=True):
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(allow_blank=True, max_length=31)

In [16]:
pprint(s_jambon.data)

{'contact': 'django@jambonsw.com',
 'description': 'Software Consulting & Training for Web and Mobile Products',
 'founded_date': '2013-01-18',
 'id': 1,
 'name': 'JamBon Software',
 'slug': 'jambon-software',
 'tags': [OrderedDict([('id', 1), ('name', 'django'), ('slug', 'django')]),
          OrderedDict([('id', 2), ('name', 'web'), ('slug', 'web')])],
 'website': 'https://www.jambonsw.com'}


In [17]:
render_json(s_jambon)

{
    "id": 1,
    "name": "JamBon Software",
    "slug": "jambon-software",
    "description": "Software Consulting & Training for Web and Mobile Products",
    "founded_date": "2013-01-18",
    "contact": "django@jambonsw.com",
    "website": "https://www.jambonsw.com",
    "tags": [
        {
            "id": 1,
            "name": "django",
            "slug": "django"
        },
        {
            "id": 2,
            "name": "web",
            "slug": "web"
        }
    ]
}


In [18]:
render_xml(s_jambon)

<?xml version="1.0" ?>
<root>
	<id>1</id>
	<name>JamBon Software</name>
	<slug>jambon-software</slug>
	<description>Software Consulting &amp; Training for Web and Mobile Products</description>
	<founded_date>2013-01-18</founded_date>
	<contact>django@jambonsw.com</contact>
	<website>https://www.jambonsw.com</website>
	<tags>
		<list-item>
			<id>1</id>
			<name>django</name>
			<slug>django</slug>
		</list-item>
		<list-item>
			<id>2</id>
			<name>web</name>
			<slug>web</slug>
		</list-item>
	</tags>
</root>



In [19]:
nl = NewsLink(
    title='JamBon Software rated best ever',
    slug='jambon-best',
    pub_date=date(2018,4,1),
    link='https://www.xkcd.com/353/',
    startup=jambon,
)

In [20]:
s_nl = NewsLinkSerializer(nl)

In [21]:
s_nl

NewsLinkSerializer(<NewsLink: JamBon Software: JamBon Software rated best ever>):
    id = IntegerField(read_only=True)
    title = CharField(max_length=63)
    slug = SlugField(max_length=63)
    pub_date = DateField()
    link = URLField(max_length=255)
    startup = StartupSerializer():
        id = IntegerField(read_only=True)
        name = CharField(max_length=31)
        slug = SlugField(allow_blank=True, max_length=31)
        description = CharField()
        founded_date = DateField()
        contact = EmailField()
        website = URLField(max_length=255)
        tags = TagSerializer(many=True):
            id = IntegerField(read_only=True)
            name = CharField(max_length=31)
            slug = SlugField(allow_blank=True, max_length=31)

In [22]:
pprint(s_nl.data)

{'id': None,
 'link': 'https://www.xkcd.com/353/',
 'pub_date': '2018-04-01',
 'slug': 'jambon-best',
 'startup': OrderedDict([('id', 1),
                         ('name', 'JamBon Software'),
                         ('slug', 'jambon-software'),
                         ('description',
                          'Software Consulting & Training for Web and Mobile '
                          'Products'),
                         ('founded_date', '2013-01-18'),
                         ('contact', 'django@jambonsw.com'),
                         ('website', 'https://www.jambonsw.com'),
                         ('tags',
                          [OrderedDict([('id', 1),
                                        ('name', 'django'),
                                        ('slug', 'django')]),
                           OrderedDict([('id', 2),
                                        ('name', 'web'),
                                        ('slug', 'web')])])]),
 'title': 'JamBon Software rated 

In [23]:
render_json(s_nl)

{
    "id": null,
    "title": "JamBon Software rated best ever",
    "slug": "jambon-best",
    "pub_date": "2018-04-01",
    "link": "https://www.xkcd.com/353/",
    "startup": {
        "id": 1,
        "name": "JamBon Software",
        "slug": "jambon-software",
        "description": "Software Consulting & Training for Web and Mobile Products",
        "founded_date": "2013-01-18",
        "contact": "django@jambonsw.com",
        "website": "https://www.jambonsw.com",
        "tags": [
            {
                "id": 1,
                "name": "django",
                "slug": "django"
            },
            {
                "id": 2,
                "name": "web",
                "slug": "web"
            }
        ]
    }
}


In [24]:
render_xml(s_nl)

<?xml version="1.0" ?>
<root>
	<id/>
	<title>JamBon Software rated best ever</title>
	<slug>jambon-best</slug>
	<pub_date>2018-04-01</pub_date>
	<link>https://www.xkcd.com/353/</link>
	<startup>
		<id>1</id>
		<name>JamBon Software</name>
		<slug>jambon-software</slug>
		<description>Software Consulting &amp; Training for Web and Mobile Products</description>
		<founded_date>2013-01-18</founded_date>
		<contact>django@jambonsw.com</contact>
		<website>https://www.jambonsw.com</website>
		<tags>
			<list-item>
				<id>1</id>
				<name>django</name>
				<slug>django</slug>
			</list-item>
			<list-item>
				<id>2</id>
				<name>web</name>
				<slug>web</slug>
			</list-item>
		</tags>
	</startup>
</root>

